In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from astropy.timeseries import LombScargle
import matplotlib as mpl


plt.rcParams.update({'figure.dpi': '300'})
path_to_res='results/'
#path_to_res='plots/big_quad_res/'




Reading LC data

In [13]:

data_0=np.array(pd.read_table(path_to_res+'lightcurve0.dat', delimiter=r"\s+", header=None))
data_45=np.array(pd.read_table(path_to_res+'lightcurve45.dat', delimiter=r"\s+", header=None))
data_90=np.array(pd.read_table(path_to_res+'lightcurve90.dat', delimiter=r"\s+", header=None))
data_180=np.array(pd.read_table(path_to_res+'lightcurve180.dat', delimiter=r"\s+", header=None))


#all down to 1 size
maxstep=np.min([len(data_0[:,0]),len(data_45[:,0]),len(data_90[:,0])])

t=data_0[:maxstep,0]*3.3e-5

data_0=data_0[:maxstep]
data_45=data_45[:maxstep]
data_90=data_90[:maxstep]
data_180=data_180[:maxstep]

In [ ]:
#filtered_0=savgol_filter(data_0[:,1], window_length=80000, polyorder=3, mode="nearest")

model = np.polyfit(t, data_0[:,1], 3)
filtered_0 = np.polyval(model, t)

plt.plot(t,data_0[:,1])
plt.plot(t,filtered_0)
plt.xlabel('t')
plt.ylabel('L')

In [ ]:
#filtered_45=savgol_filter(data_45[:,1], window_length=80000, polyorder=3, mode="nearest")

model = np.polyfit(t, data_45[:,1], 3)
filtered_45 = np.polyval(model, t)


plt.plot(t,data_45[:,1])
plt.plot(t,filtered_45)
plt.xlabel('t')
plt.ylabel('L')

In [ ]:
#filtered_90=savgol_filter(data_90[:,1], window_length=80000, polyorder=3, mode="nearest")

model = np.polyfit(t, data_90[:,1], 3)
filtered_90 = np.polyval(model, t)


plt.plot(t,data_90[:,1])
plt.plot(t,filtered_90)
plt.xlabel('t')
plt.ylabel('L')

In [ ]:
#filtered_180=savgol_filter(data_180[:,1], window_length=80000, polyorder=3, mode="nearest")

model = np.polyfit(t, data_180[:,1], 3)
filtered_180 = np.polyval(model, t)


plt.plot(t,data_180[:,1])
plt.plot(t,filtered_180)
plt.xlabel('t')
plt.ylabel('L')

Removing the trend

In [18]:

lc_0=data_0[:,1]-filtered_0
lc_45=data_45[:,1]-filtered_45
lc_90=data_90[:,1]-filtered_90
lc_180=data_180[:,1]-filtered_180

Splitting arrays into parts

In [19]:
N_splits=10

t_split=np.array_split(t,N_splits)
lc_0_split=np.array_split(lc_0,N_splits)
lc_45_split=np.array_split(lc_45,N_splits)
lc_90_split=np.array_split(lc_90,N_splits)
lc_180_split=np.array_split(lc_180,N_splits)

Computing the power spectra

In [ ]:
max_freq=200

freq_0=[]
pow_0=[]

freq_45=[]
pow_45=[]

freq_90=[]
pow_90=[]


freq_180=[]
pow_180=[]

for num,t_el in enumerate(t_split):
    fr, po = LombScargle(t_el,lc_0_split[num]).autopower(samples_per_peak=10,nyquist_factor=1, maximum_frequency=max_freq)
    freq_0.append(fr)
    pow_0.append(po)
    fr, po = LombScargle(t_el,lc_45_split[num]).autopower(samples_per_peak=10,nyquist_factor=1, maximum_frequency=max_freq)
    freq_45.append(fr)
    pow_45.append(po)
    fr, po = LombScargle(t_el,lc_90_split[num]).autopower(samples_per_peak=10,nyquist_factor=1, maximum_frequency=max_freq)
    freq_90.append(fr)
    pow_90.append(po)
    fr, po = LombScargle(t_el,lc_180_split[num]).autopower(samples_per_peak=10,nyquist_factor=1, maximum_frequency=max_freq)
    freq_180.append(fr)
    pow_180.append(po)

print(len(fr))

In [21]:
start_times = [item[0] for item in t_split]
start_times.append(t_split[-1][-1])

Computing omegas. 
If your omega.dat is big the reading process might take a while

In [22]:
n_rows = 2500 #temp fix for big files
skip = np.arange(n_rows)
skip = np.delete(skip, np.arange(0, n_rows, 5))


In [33]:
data_omega=pd.read_table(path_to_res+'omega.dat', delimiter=r"\s+", header=None, skiprows = skip)
face_centers=pd.read_table(path_to_res+'face_centers.dat', header=None, delimiter=r"\s+")
face_centers=np.array(face_centers)
theta_fc=np.arccos(face_centers[:,2]/np.linalg.norm(face_centers, axis=1))
mask=np.logical_and(theta_fc > 0.03*np.pi, theta_fc < 0.996 * np.pi)


maxstep=len(data_omega.loc[:,0])
t_omega=data_omega.loc[:,0]
om_avg=[]

for i in range(maxstep):
    om_avg.append(np.average(data_omega.loc[i,1:][mask]))

om_avg=np.array(om_avg)
om_avg/=(3.3e-5*2*np.pi)
t_omega*=3.3e-5


Same as before, but here we compute the cyclone frequency via finding the pressure minimum.
It may also take a while if p.dat is a large file.

In [34]:
n_rows = 2500 #temp fix for big files
skip = np.arange(n_rows)
skip = np.delete(skip, np.arange(0, n_rows, 5))


In [35]:
data_curl=pd.read_table(path_to_res+'p.dat', delimiter=r"\s+", header=None, skiprows = skip)


In [36]:
face_centers=pd.read_table(path_to_res+'face_centers.dat', header=None, delimiter=r"\s+")
fc=np.array(face_centers)
phi_fc=np.arctan2(fc[:,1]/np.linalg.norm(fc, axis=1),fc[:,0]/np.linalg.norm(fc, axis=1))
maxstep=len(data_curl.loc[:,0])
theta_fc=np.arccos(fc[:,2]/np.linalg.norm(fc, axis=1))
t=data_curl.loc[:,0]*3.3e-5
phi_cyclon=[]
theta_cyclon=[]
for i in range(maxstep):
    phi_cyclon.append(phi_fc[np.argmin(data_curl.loc[i,1:98304])])
    theta_cyclon.append(theta_fc[np.argmin(data_curl.loc[i,1:98304])])


In [38]:
phi_cyclon=np.array(phi_cyclon)
theta_cyclon=np.array(theta_cyclon)
t=np.array(t)
der=(phi_cyclon[1:]-phi_cyclon[:-1])/(t[1:]-t[:-1])

der2=(theta_cyclon[1:]-theta_cyclon[:-1])/(t[1:]-t[:-1])

der_f=np.delete(der,np.where(der<0))
t_f=np.delete(t[1:],np.where(der<0))
der2_f=np.delete(der2,np.where(der<0))
#t_omega_cycl=t_f[160:]
#om_avg_cycl=der_f[160:]/(2*np.pi)
t_omega_cycl=t_f
om_avg_cycl=der_f/(2*np.pi)

Plots

In [29]:
max_freq+=1

In [ ]:
colorm = plt.get_cmap('viridis')
plt.rcParams.update({'font.size': 22})

#for whatever reason min and np.min do not want to work
#with an array consisting of multiple arrays of different lengths
#filling small ones with zeros will ruin some things btw

mins=[]
maxs=[]
for po in pow_0: 
    mins.append(min(po))
    maxs.append(max(po))

min_gl=min(mins)
max_gl=max(maxs)

norm = mpl.colors.Normalize(vmin=min_gl, vmax=max_gl)



fig, ax = plt.subplots(figsize=(10, 10), layout='constrained', nrows=2,height_ratios=[15,1])
plt.subplots_adjust(hspace=10)
fig.suptitle(r'Dynamic power spectrum, $\theta_{\rm obs}=90$°')
ax[0].set_xlabel(r'Freq, Hz', fontsize=25)
ax[0].set_ylabel(r'Simulation time, s', fontsize=25)



for ind, freq_arr in enumerate(freq_0):
    for el_num, el in enumerate(freq_0[ind]):
        
        y=[start_times[ind],start_times[ind],start_times[ind+1],start_times[ind+1]]
        if(el_num+1!=len(freq_0[ind])):
            x=[freq_0[ind][el_num],freq_0[ind][el_num+1],freq_0[ind][el_num+1],freq_0[ind][el_num]]
        else:
            x=[freq_0[ind][el_num],max_freq,max_freq,freq_0[ind][el_num]]
        ax[0].fill(x, y,facecolor=colorm(norm(pow_0[ind][el_num])),edgecolor =colorm(norm(pow_0[ind][el_num])))



ax[0].plot(om_avg_cycl[::6],t_omega_cycl[::6], color='#ff4538',linewidth=3, label=r"$\nu_{\rm cycl}$",linestyle='solid')
ax[0].plot(om_avg[::21],t_omega[::21], color='tab:orange',linewidth=3, label=r"$\nu$", marker='.', markersize=10)
ax[0].plot(2*om_avg[::21],t_omega[::21], color='#f2ff38',linewidth=3, label=r"$2 \nu$", marker='D', markersize=5)
#ax[0].plot(2*om_avg_cycl,t_omega_cycl, color='tab:red',linewidth=3, label=r"$2 \nu_{\rm cycl}$")
ax[0].legend(loc=4)


fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colorm),cax=ax[1], orientation='horizontal', label="Power")
fig.savefig('plots/dymamic_power_spec_0.png', bbox_inches='tight')


In [ ]:
colorm = plt.get_cmap('viridis')
#max_freq+=1
plt.rcParams.update({'font.size': 22})

#for whatever reason min and np.min do not want to work
#with an array consisting of multiple arrays of different lengths
#filling small ones with zeros will ruin some things btw

mins=[]
maxs=[]
for po in pow_45: 
    mins.append(min(po))
    maxs.append(max(po))

min_gl=min(mins)
max_gl=max(maxs)

norm = mpl.colors.Normalize(vmin=min_gl, vmax=max_gl)



fig, ax = plt.subplots(figsize=(10, 10), layout='constrained', nrows=2,height_ratios=[15,1])
plt.subplots_adjust(hspace=10)
fig.suptitle(r'Dynamic power spectrum, $\theta_{\rm obs}=45$°')
ax[0].set_xlabel(r'Freq, Hz', fontsize=25)
ax[0].set_ylabel(r'Simulation time, s', fontsize=25)



for ind, freq_arr in enumerate(freq_45):
    for el_num, el in enumerate(freq_45[ind]):
        
        y=[start_times[ind],start_times[ind],start_times[ind+1],start_times[ind+1]]
        if(el_num+1!=len(freq_45[ind])):
            x=[freq_45[ind][el_num],freq_45[ind][el_num+1],freq_45[ind][el_num+1],freq_45[ind][el_num]]
        else:
            x=[freq_45[ind][el_num],max_freq,max_freq,freq_45[ind][el_num]]
        ax[0].fill(x, y,facecolor=colorm(norm(pow_45[ind][el_num])),edgecolor =colorm(norm(pow_45[ind][el_num])))



ax[0].plot(om_avg_cycl[::6],t_omega_cycl[::6], color='#ff4538',linewidth=3, label=r"$\nu_{\rm cycl}$",linestyle='solid')
ax[0].plot(om_avg[::21],t_omega[::21], color='tab:orange',linewidth=3, label=r"$\nu$", marker='.', markersize=10)
ax[0].plot(2*om_avg[::21],t_omega[::21], color='#f2ff38',linewidth=3, label=r"$2 \nu$", marker='D', markersize=5)
#ax[0].plot(2*om_avg_cycl,t_omega_cycl, color='tab:red',linewidth=3, label=r"$2 \nu_{\rm cycl}$")
ax[0].legend(loc=4)



fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colorm),cax=ax[1], orientation='horizontal', label="Power")
fig.savefig('plots/dymamic_power_spec_45.png', bbox_inches='tight')


In [ ]:
colorm = plt.get_cmap('viridis')
#max_freq+=1
plt.rcParams.update({'font.size': 22})

#for whatever reason min and np.min do not want to work
#with an array consisting of multiple arrays of different lengths
#filling small ones with zeros will ruin some things btw

mins=[]
maxs=[]

for po in pow_90:
    mins.append(min(po))
    maxs.append(max(po))

min_gl=min(mins)
max_gl=max(maxs)

norm = mpl.colors.Normalize(vmin=min_gl, vmax=max_gl)



fig, ax = plt.subplots(figsize=(10, 10), layout='constrained', nrows=2,height_ratios=[15,1])
plt.subplots_adjust(hspace=10)
fig.suptitle(r'Dynamic power spectrum, $\theta_{\rm obs}=0$° (North pole)')
ax[0].set_xlabel(r'Freq, Hz', fontsize=25)
ax[0].set_ylabel(r'Simulation time, s', fontsize=25)



for ind, freq_arr in enumerate(freq_90):
    for el_num, el in enumerate(freq_90[ind]):
        
        y=[start_times[ind],start_times[ind],start_times[ind+1],start_times[ind+1]]
        if(el_num+1!=len(freq_90[ind])):
            x=[freq_90[ind][el_num],freq_90[ind][el_num+1],freq_90[ind][el_num+1],freq_90[ind][el_num]]
        else:
            x=[freq_90[ind][el_num],max_freq,max_freq,freq_90[ind][el_num]]
        ax[0].fill(x, y,facecolor=colorm(norm(pow_90[ind][el_num])),edgecolor =colorm(norm(pow_90[ind][el_num])))



ax[0].plot(om_avg_cycl[::6],t_omega_cycl[::6], color='#ff4538',linewidth=3, label=r"$\nu_{\rm cycl}$",linestyle='solid')
ax[0].plot(om_avg[::21],t_omega[::21], color='tab:orange',linewidth=3, label=r"$\nu$", marker='.', markersize=10)
ax[0].plot(2*om_avg[::21],t_omega[::21], color='#f2ff38',linewidth=3, label=r"$2 \nu$", marker='D', markersize=5)
#ax[0].plot(2*om_avg_cycl,t_omega_cycl, color='tab:red',linewidth=3, label=r"$2 \nu_{\rm cycl}$")
ax[0].legend(loc=4)



fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colorm),cax=ax[1], orientation='horizontal', label="Power")
fig.savefig('plots/dymamic_power_spec_90.png', bbox_inches='tight')


In [ ]:
colorm = plt.get_cmap('viridis')
#max_freq+=1
plt.rcParams.update({'font.size': 22})

#for whatever reason min and np.min do not want to work
#with an array consisting of multiple arrays of different lengths
#filling small ones with zeros will ruin some things btw

mins=[]
maxs=[]
for po in pow_180: 
    mins.append(min(po))
    maxs.append(max(po))

min_gl=min(mins)
max_gl=max(maxs)

norm = mpl.colors.Normalize(vmin=min_gl, vmax=max_gl)



fig, ax = plt.subplots(figsize=(10, 10), layout='constrained', nrows=2,height_ratios=[15,1])
plt.subplots_adjust(hspace=10)
fig.suptitle(r'Dynamic power spectrum, $\theta_{\rm obs}=180$° (South pole)')
ax[0].set_xlabel(r'Freq, Hz', fontsize=25)
ax[0].set_ylabel(r'Simulation time, s', fontsize=25)



for ind, freq_arr in enumerate(freq_180):
    for el_num, el in enumerate(freq_180[ind]):
        
        y=[start_times[ind],start_times[ind],start_times[ind+1],start_times[ind+1]]
        if(el_num+1!=len(freq_180[ind])):
            x=[freq_180[ind][el_num],freq_180[ind][el_num+1],freq_180[ind][el_num+1],freq_180[ind][el_num]]
        else:
            x=[freq_180[ind][el_num],max_freq,max_freq,freq_180[ind][el_num]]
        ax[0].fill(x, y,facecolor=colorm(norm(pow_180[ind][el_num])),edgecolor =colorm(norm(pow_180[ind][el_num])))


ax[0].plot(om_avg_cycl[::6],t_omega_cycl[::6], color='#ff4538',linewidth=3, label=r"$\nu_{\rm cycl}$",linestyle='solid')
ax[0].plot(om_avg[::21],t_omega[::21], color='tab:orange',linewidth=3, label=r"$\nu$", marker='.', markersize=10)
ax[0].plot(2*om_avg[::21],t_omega[::21], color='#f2ff38',linewidth=3, label=r"$2 \nu$", marker='D', markersize=5)
#ax[0].plot(2*om_avg_cycl,t_omega_cycl, color='tab:red',linewidth=3, label=r"$2 \nu_{\rm cycl}$")
ax[0].legend(loc=4)




fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=colorm),cax=ax[1], orientation='horizontal', label="Power")
fig.savefig('plots/dymamic_power_spec_270.png', bbox_inches='tight')
